# Team win history GRU

Choose two teams, the history will first be displayed as (for example) 0,1,0,..,1,0 etc.
Then these are compiled and a GRU is made to predict the probability of future win of that team against a challenger...

In [1]:
# Python imports
import pandas as pd
import numpy as np

In [2]:
dat = pd.read_csv('matchdata.csv',encoding = "ISO-8859-1")


/home/max/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (82,83,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Need a function that pulls out only a certain teams data:

def team_data(team): # probably by index...
    df = dat.loc[(dat.ix[:,3] == team) |  (dat.ix[:,4] == team) ]
    return(df)


In [284]:
def win_series(team1,team2):
    wins = []
    losses = []
    
    # find only games where team 1 and team 2 are playing
    df = dat.loc[((dat.ix[:,3] == team1) |  (dat.ix[:,4] == team1) ) & ((dat.ix[:,3] == team2) |  (dat.ix[:,4] == team2) )]
    df = df.reset_index(drop = True) 
    # now for each row find if they won or lost: winning team == team1
    for i in range(len(df)):

        if (df.ix[i,5] == team1):
            # they won
            wins.append(1)
            losses.append(0)
        elif (df.ix[i,5] == team2):
            # they lost
            wins.append(0)
            losses.append(1)
        else:
            # can't find them
            wins.append('?')
    
    # now reverse the win vector to be oldest to newest
    wins = np.flipud(wins)
    ins= wins[:len(wins)-1]
    targs = wins[1:len(wins)] 
    lins = np.array(losses[:len(wins)-1]) 
    ltargs = np.array(losses[1:len(wins)])
    
    return ins, targs, lins,ltargs
    

In [285]:
''' sequence padding code '''
from __future__ import absolute_import
# -*- coding: utf-8 -*-
import random
from six.moves import range


def pad_sequences(sequences, maxlen=None, dtype='int32',
                  padding='pre', truncating='pre', value=0.):
    '''Pads each sequence to the same length:
    the length of the longest sequence.

    If maxlen is provided, any sequence longer
    than maxlen is truncated to maxlen.
    Truncation happens off either the beginning (default) or
    the end of the sequence.

    Supports post-padding and pre-padding (default).

    # Arguments
        sequences: list of lists where each element is a sequence
        maxlen: int, maximum length
        dtype: type to cast the resulting sequence.
        padding: 'pre' or 'post', pad either before or after each sequence.
        truncating: 'pre' or 'post', remove values from sequences larger than
            maxlen either in the beginning or in the end of the sequence
        value: float, value to pad the sequences to the desired value.

    # Returns
        x: numpy array with dimensions (number_of_sequences, maxlen)
    '''
    lengths = [len(s) for s in sequences]

    nb_samples = len(sequences)
    if maxlen is None:
        maxlen = np.max(lengths)

    # take the sample shape from the first non empty sequence
    # checking for consistency in the main loop below.
    sample_shape = tuple()
    for s in sequences:
        if len(s) > 0:
            sample_shape = np.asarray(s).shape[1:]
            break

    x = (np.ones((nb_samples, maxlen) + sample_shape) * value).astype(dtype)
    for idx, s in enumerate(sequences):
        if len(s) == 0:
            continue  # empty list was found
        if truncating == 'pre':
            trunc = s[-maxlen:]
        elif truncating == 'post':
            trunc = s[:maxlen]
        else:
            raise ValueError('Truncating type "%s" not understood' % truncating)

        # check `trunc` has expected shape
        trunc = np.asarray(trunc, dtype=dtype)
        if trunc.shape[1:] != sample_shape:
            raise ValueError('Shape of sample %s of sequence at position %s is different from expected shape %s' %
                             (trunc.shape[1:], idx, sample_shape))

        if padding == 'post':
            x[idx, :len(trunc)] = trunc
        elif padding == 'pre':
            x[idx, -len(trunc):] = trunc
        else:
            raise ValueError('Padding type "%s" not understood' % padding)
    return x


In [286]:
X, T, XL , TL = win_series('EnVyUs','fnatic')
X1, T1, XL1 , TL1 = win_series('EnVyUs','Natus Vincere')
X2, T2, XL2 , TL2 = win_series('EnVyUs','Luminosity')
X3, T3, XL3 , TL3 = win_series('EnVyUs','Virtus.pro')
X4, T4, XL4 , TL4 = win_series('EnVyUs','TSM')
X5, T5, XL5 , TL5 = win_series('EnVyUs','Astralis')
X6, T6, XL6 , TL6 = win_series('EnVyUs','NiP')


# now zero pad all to be the same shape:



# combine into an array:

ins =  pad_sequences(np.array([X.tolist(),X1.tolist(),X2.tolist(),X3.tolist(),X4.tolist(),X5.tolist(),X6.tolist()]) )
targs = pad_sequences(np.array([T.tolist(),T1.tolist(),T2.tolist(),T3.tolist(),T4.tolist(),T5.tolist(),T6.tolist()]) )

lins =  pad_sequences(np.array([XL.tolist(),XL1.tolist(),XL2.tolist(),XL3.tolist(),XL4.tolist(),XL5.tolist(),XL6.tolist()]) )

ltargs = pad_sequences(np.array([TL.tolist(),TL1.tolist(),TL2.tolist(),TL3.tolist(),TL4.tolist(),TL5.tolist(),TL6.tolist()]) )



In [287]:
''' split into testing and training sets '''
len(ins)
split = 0.7
Ntrain = int(round(len(ins)*split,0))

train_ins = ins[:Ntrain]
test_ins = ins[Ntrain:]
train_T = targs[:Ntrain]
test_T = targs[Ntrain:]

In [288]:
train_ins

array([[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
        0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0,
        1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1,
        0, 1, 1, 1, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 1, 0, 0, 1, 0]], dtype=int32)

In [289]:
# Run the model of GRU
# Keras imports 
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM,GRU, Embedding

In [290]:
in_neurons = 1
hidden_neurons = 1
 
model = Sequential()  
model.add(GRU(output_dim=hidden_neurons, input_dim= in_neurons, return_sequences=True, consume_less='gpu')) 
model.add(GRU(output_dim=hidden_neurons, input_dim= hidden_neurons, return_sequences=True, consume_less='gpu')) 
model.compile(loss="binary_crossentropy", optimizer= 'adam' ) 

In [ ]:
''' train this fella '''
model.fit(np.reshape(train_ins,(len(train_ins),52,1) ), np.reshape(train_T,(len(train_ins),52,1) ), batch_size=1, nb_epoch = 500, validation_split=0.1)  


Train on 4 samples, validate on 1 samples
Epoch 1/500
4/4 [==============================] - 0s - loss: 0.5619 - val_loss: 0.3815
Epoch 2/500
4/4 [==============================] - 0s - loss: 0.5619 - val_loss: 0.3812
Epoch 3/500
4/4 [==============================] - 0s - loss: 0.5618 - val_loss: 0.3811
Epoch 4/500
4/4 [==============================] - 0s - loss: 0.5617 - val_loss: 0.3809
Epoch 5/500
4/4 [==============================] - 0s - loss: 0.5616 - val_loss: 0.3807
Epoch 6/500
4/4 [==============================] - 0s - loss: 0.5615 - val_loss: 0.3806
Epoch 7/500
4/4 [==============================] - 0s - loss: 0.5615 - val_loss: 0.3805
Epoch 8/500
4/4 [==============================] - 0s - loss: 0.5614 - val_loss: 0.3804
Epoch 9/500
4/4 [==============================] - 0s - loss: 0.5614 - val_loss: 0.3801
Epoch 10/500
4/4 [==============================] - 0s - loss: 0.5613 - val_loss: 0.3799
Epoch 11/500
4/4 [==============================] - 0s - loss: 0.5612 - val_l

In [282]:
Tshape = np.shape(train_ins)[1]
predicted = model.predict(np.reshape(test_ins,(np.shape(test_T)[0],Tshape,1)))

preds = pd.DataFrame()
preds['Real'] = np.reshape(test_T[1],(Tshape) )
preds['Predicted'] = np.reshape(predicted[1],(Tshape) )
print(preds)

    Real  Predicted
0      0  -0.023531
1      0  -0.012929
2      0  -0.013260
3      0  -0.012094
4      0  -0.011867
5      0  -0.011678
6      0  -0.011608
7      0  -0.011570
8      0  -0.011554
9      0  -0.011545
10     0  -0.011541
11     0  -0.011539
12    -1   0.056036
13    -1  -0.037172
14     1  -0.068246
15    -1   0.055354
16    -1  -0.047476
17     1  -0.072815
18     1   0.054957
19     1   0.053395
20     1   0.062464
21    -1   0.062969
22     1  -0.020749
23    -1   0.060462
24    -1  -0.036619
25    -1  -0.067337
26     1  -0.094572
27     1   0.054746
28     1   0.051846
29     1   0.062352
30    -1   0.062848
31     1  -0.020830
32     1   0.060454
33     1   0.057426
34    -1   0.063124
35    -1  -0.021545
36     1  -0.060305
37     1   0.055978
38     1   0.054335
39    -1   0.062597
40    -1  -0.022350
41    -1  -0.060743
42     1  -0.089847
43    -1   0.054879
44    -1  -0.051485
45    -1  -0.074615
46    -1  -0.099579
47    -1  -0.112396
48    -1  -0.120486


In [227]:
INP = win_series('EnVyUs','G2')
predicted = model.predict(np.reshape(np.array(INP) , (2,len(INP[0]),1)) )[0]


In [228]:
predicted

array([[ 0.00726403],
       [ 0.01320932],
       [ 0.54285729],
       [ 0.54242665],
       [ 0.54244822],
       [ 0.54244715],
       [ 0.49628255],
       [ 0.45058441],
       [ 0.40618879],
       [ 0.36383224],
       [ 0.32410753],
       [ 0.5488987 ],
       [ 0.54221153],
       [ 0.54245901],
       [ 0.54244661],
       [ 0.49628198]], dtype=float32)

In [234]:
INP = win_series('EnVyUs','HellRaisers')

predicted = model.predict(np.reshape(np.array(INP) , (2,len(INP[0]),1)) )[0]
TTshape = len(predicted)
preds = pd.DataFrame()
preds['Real'] = np.reshape(INP[0],(TTshape) )
preds['Predicted'] = np.reshape(predicted,(TTshape) )
print(preds)

    Real  Predicted
0      1   0.542201
1      1   0.542459
2      1   0.542447
3      1   0.542447
4      1   0.542447
5      0   0.496283
6      0   0.450584
7      1   0.546245
8      1   0.542259
9      0   0.496094
10     1   0.544562
11     1   0.542341
12     1   0.542453
13     1   0.542447
14     1   0.542447
